# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering
algorithm to group pixels by similar spectral signatures. **k-means** is
an **exploratory** method for finding patterns in data. Because it is
unsupervised, you don’t need any training data for the model. You also
can’t measure how well it “performs” because the clusters will not
correspond to any particular land cover class. However, we expect at
least some of the clusters to be identifiable as different types of land
cover.

You will use the [harmonized Sentinal/Landsat multispectral
dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf).
You can access the data with an [Earthdata
account](https://www.earthdata.nasa.gov/learn/get-started) and the
[`earthaccess` library from
NSIDC](https://github.com/nsidc/earthaccess):

## STEP 1: SET UP

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Import all libraries you will need for this analysis</li>
<li>Configure GDAL parameters to help avoid connection errors:
<code>python      os.environ["GDAL_HTTP_MAX_RETRY"] = "5"      os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"</code></li>
</ol></div></div>

In [1]:
import os
import pathlib
import pickle
import re
import warnings

import cartopy.crs as ccrs
import earthaccess
import earthpy as et
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
from tqdm.notebook import tqdm
import xarray as xr
from shapely.geometry import Polygon
from sklearn.cluster import KMeans
import zipfile

os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

warnings.simplefilter('ignore')

c:\Users\riede\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Make data directory
data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'mississippi-delta'
    )

os.makedirs(data_dir, exist_ok=True)

data_dir

'C:\\Users\\riede\\earth-analytics\\data\\mississippi-delta'

Below you can find code for a caching **decorator** which you can use in
your code. To use the decorator:

``` python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the
`do_something()` function, and only run the code if the results don’t
already exist. To override the caching, for example temporarily after
making changes to your code, set `override=True`. Note that to use the
caching decorator, you must write your own function to perform each
task!

In [3]:
def cached(func_key, override=False):
    """
    A decorator to cache function results
    
    Parameters
    ==========
    key: str
      File basename used to save pickled results
    override: bool
      When True, re-compute even if the results are already stored
    """
    def compute_and_cache_decorator(compute_function):
        """
        Wrap the caching function
        
        Parameters
        ==========
        compute_function: function
          The function to run and cache results
        """
        def compute_and_cache(*args, **kwargs):
            """
            Perform a computation and cache, or load cached result.
            
            Parameters
            ==========
            args
              Positional arguments for the compute function
            kwargs
              Keyword arguments for the compute function
            """
            # Add an identifier from the particular function call
            if 'cache_key' in kwargs:
                key = '_'.join((func_key, kwargs['cache_key']))
            else:
                key = func_key

            path = os.path.join(
                et.io.HOME, et.io.DATA_NAME, 'jars', f'{key}.pickle')
            
            # Check if the cache exists already or override caching
            if not os.path.exists(path) or override:
                # Make jars directory if needed
                os.makedirs(os.path.dirname(path), exist_ok=True)
                
                # Run the compute function as the user did
                result = compute_function(*args, **kwargs)
                
                # Pickle the object
                with open(path, 'wb') as file:
                    pickle.dump(result, file)
            else:
                # Unpickle the object
                with open(path, 'rb') as file:
                    result = pickle.load(file)
                    
            return result
        
        return compute_and_cache
    
    return compute_and_cache_decorator

## STEP 2: STUDY SITE

For this analysis, you will use a watershed from the [Water Boundary
Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products),
HU12 watersheds (WBDHU12.shp).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Download the Water Boundary Dataset for region 8 (Mississippi)</li>
<li>Select watershed 080902030506</li>
<li>Generate a site map of the watershed</li>
</ol>
<p>Try to use the <strong>caching decorator</strong></p></div></div>

We chose this watershed because it covers parts of New Orleans and is
near the Mississippi Delta. Deltas are boundary areas between the land
and the ocean, and as a result tend to contain a rich variety of
different land cover and land use types.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-response"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>Write a 2-3 sentence <strong>site description</strong> (with
citations) of this area that helps to put your analysis in context.</p></div></div>

**SEE SITE DESCRIPTION BELOW SITE MAP**

* Helpful links:

    * URL for WBD region 8: https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/WBD_08_HU2_Shape.zip

    * Documentation for et.data.get_data: https://earthpy.readthedocs.io/en/latest/gallery_vignettes/get_data.html

    * Documentation for .dissolve(): https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html

<span style="color: purple;">

#### STEP 2a. Download watershed boundary data for region 8

</span>

In [4]:
# call cached decorator
@cached('wbd', override=False)
# add function to apply decorator abilities to
def read_wbd_file(wbd_filename, huc_level, cache_key):
    # Define wbd_url
    wbd_url = (
        "https://prd-tnm.s3.amazonaws.com"
        "/StagedProducts/Hydrography/WBD/HU2/Shape/"
        f"{wbd_filename}.zip")
    # et.data.get_data from earthpy library to download data from a zip file
    # can also download Earthpy Dataset Subsets w/ et.data.get_data 
    wbd_dir = et.data.get_data(url=wbd_url)
                  
    # Read desired data
    wbd_path = os.path.join(wbd_dir, 'Shape', f'WBDHU{huc_level}.shp')
    # engine is the library used to read the file
    wbd_gdf = gpd.read_file(wbd_path, engine='pyogrio')
    return wbd_gdf

# define huc_level
huc_level = 12

# create wbd_total_gdf 
wbd_total_gdf = read_wbd_file(
    # filename found from URL noted above code cell
    wbd_filename = 'WBD_08_HU2_Shape',
    # huc_level defined above
    huc_level = huc_level,
    # cache_key is a kwarg for cached decorator
    cache_key=f'hu{huc_level}'
)

# check wbd_total_gdf
wbd_total_gdf.head()


,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID,geometry
0,{8AFB1AF9-7296-4303-89DE-14CD073B859A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"535297,540579",29441.81,119.15,LA,...,Gourd Bayou-Youngs Bayou,S,"LE,ID,DD",080500011308,0.0,0.0,NaN,NaN,1,"POLYGON ((-92.00021 32.53586, -91.99994 32.535..."
1,{916A17A6-B4A0-4FD7-9BB8-FFD1936B15B2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,535512,11406.67,46.16,LA,...,Hams Creek,S,ID,080802050104,0.0,0.0,NaN,NaN,2,"POLYGON ((-93.37574 30.58982, -93.3747 30.5891..."
2,{493C7EC1-2F1C-4B84-AFFB-6F6868A9868E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547190,559640",29138.21,117.92,LA,...,Caney Creek-Bayou D'Arbonne,S,NM,080402060503,0.0,0.0,NaN,NaN,3,"POLYGON ((-93.07761 32.88752, -93.07784 32.887..."
3,{49A3C087-B460-4F97-9D99-78CBB675248B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77417,78285",17759.39,71.87,AR,...,L'Aigle Creek-Saline River,S,NM,080402020206,0.0,0.0,NaN,NaN,4,"POLYGON ((-92.08947 33.29383, -92.0897 33.2938..."
4,{0FB41498-11EA-4AB1-AF05-E2A8E5E2E274},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,1628466,98564.62,398.88,LA,...,West Cote Blanche Bay,W,NM,080801030800,0.0,0.0,NaN,NaN,5,"POLYGON ((-91.62408 29.73947, -91.62195 29.737..."


<span style="color: purple;">

#### STEP 2b. Select watershed of interest from region 8, 080902030506

</span>

In [5]:
# see all column names of the wbd_total_gdf
wbd_total_gdf.columns

Index(['tnmid', 'metasource', 'sourcedata', 'sourceorig', 'sourcefeat',
       'loaddate', 'referenceg', 'areaacres', 'areasqkm', 'states', 'huc12',
       'name', 'hutype', 'humod', 'tohuc', 'noncontrib', 'noncontr_1',
       'shape_Leng', 'shape_Area', 'ObjectID', 'geometry'],
      dtype='object')

In [6]:
# select the 080902030506 rows from the wbd_total_gdf
wbd_gdf = (
    # from the huc12 column...
    wbd_total_gdf[wbd_total_gdf[f'huc{huc_level}']
    # select only the rows for 080902030506
    .isin(['080902030506'])]
    # dissolve all geometries to a single geometric feature
    # and aggreagate all rows of data in a group
    .dissolve()
)

# check wbd_gdf
wbd_gdf

,geometry,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,...,huc12,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID
0,"POLYGON ((-89.97047 29.74687, -89.96593 29.750...",{E942B72E-599E-48F5-908A-EA5265701C14},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"536881,539539",37355.86,151.17,...,080902030506,Manuel Canal-Spanish Lake,D,GC,080902030508,0.0,0.0,NaN,NaN,2560


In [7]:
print(f"Watershed 080902030506 is in {wbd_gdf['states']}")

Watershed 080902030506 is in 0    LA
Name: states, dtype: object


<span style="color: purple;">

#### STEP 2c. Create site map of watershed 080902030506 in Louisiana, USA & describe watershed

</span>

In [8]:
# create site map with hvplot
(
    wbd_gdf
    .hvplot(
        # geographic plot, lat/lon axes
        geo=True,
        # overlay plot on EsriImagery tiles
        tiles='EsriImagery',
        # fill color
        color='white',
        # transparency of fill color
        alpha=0.4,
        # line color
        line_color='red',
        # define title
        title='080902030506 Watershed Boundary in Louisiana, USA',
        # define size of plot
        frame_width=400
    )
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

<span style="color: maroon;">

**SITE DESCRIPTION:**

Watershed 080902030506 is in Louisiana, USA, southeast of New Orleans. According to Naitve Land Digital, the watershed is on the Indigenous territories of the [Chahta Yakni (Choctaw)](https://native-land.ca/maps/territories/chahta-choctaw), [Houma](https://native-land.ca/maps/territories/houma-2), and [Chitimacha](https://native-land.ca/maps/territories/chitimacha-2) people. The Mississippi river runs north to south on the east side of the waterhsed. Lake Lery is on the north east side of the watershed. This watershed is in Louisiana's [8th of 9 watersheds](https://watershed.la.gov/watershed-regions). According to [Louisiana's current Region 8 flood risk report](https://d10zxfp0rexahe.cloudfront.net/docs/Region-8-PDF.pdf), this region of Louisiana is subject to 3 main different types of flooding: fluvial/river, costal/surge and tidal, and pluvial/rainfall-induced flash floods and urban flooding. Watershed 080902030506 is also classified as medium-high in the CDC social vulnerability index based on socioeconimic status, household composition and disability, minority status and language, and housing and transportation. 

</span>

Extra links, may or may not use:

how's my waterway (water condition info): https://mywaterway.epa.gov/community/080902030506/overview

about the wbd: https://www.usgs.gov/national-hydrography/watershed-boundary-dataset


## STEP 3: MULTISPECTRAL DATA

### Search for data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Log in to the <code>earthaccess</code> service using your Earthdata
credentials:
<code>python      earthaccess.login(persist=True)</code></li>
<li>Modify the following sample code to search for granules of the
HLSL30 product overlapping the watershed boundary from May to October
2023 (there should be 76 granules):
<code>python      results = earthaccess.search_data(          short_name="...",          cloud_hosted=True,          bounding_box=tuple(gdf.total_bounds),          temporal=("...", "..."),      )</code></li>
</ol></div></div>

HLSL30 info from the NASA worldview site:

"The Harmonized Landsat Sentinel-2 (HLS) project brings us 30 meter resolution true color surface reflectance imagery from the Operational Land Imager (OLI) instrument aboard the NASA/USGS Landsat 8 and 9 satellites, and the Multi-Spectral Instrument (MSI) aboard the European Space Agency (ESA) Sentinel-2A and Sentinel-2B satellites."

In [9]:
# Log in to earthaccess
earthaccess.login(strategy="interactive", persist=True)

In [10]:
# Search for HLS tiles
wbd_hls_results = earthaccess.search_data(
    # dataset short name
    short_name='HLSL30',
    # the bounding box is the waterhsed boundary
    bounding_box=tuple(wbd_gdf.total_bounds),
    # temporal bounds from May - Oct 2023
    temporal=('2023-05-01', '2023-10-31')
)
wbd_hls_results


[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -90.07372506, 'Latitude': 28.80444525}, {'Longitude': -89.57473392, 'Latitude': 28.81489352}, {'Longitude': -89.30816409, 'Latitude': 29.81030157}, {'Longitude': -90.10353675, 'Latitude': 29.79400715}, {'Longitude': -90.07372506, 'Latitude': 28.80444525}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2023-05-04T16:31:32.101Z', 'EndingDateTime': '2023-05-04T16:31:55.992Z'}}
 Size(MB): 87.65197467803955
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B07.tif', 'https://data.lp

In [11]:
# Open earthaccess results
# There are 15 bands per tile, 1320/15 = 88 tiles
# commenting this out, don't think these need to be opened right now
# wbd_hls_files = earthaccess.open(wbd_hls_results)

In [12]:
# add the wbd_hls_results go a geodataframe
wbd_hls_gdf = gpd.GeoDataFrame(wbd_hls_results)
wbd_hls_gdf

,meta,umm,size
0,"{'concept-type': 'granule', 'concept-id': 'G26...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,87.651975
1,"{'concept-type': 'granule', 'concept-id': 'G26...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,150.901505
2,"{'concept-type': 'granule', 'concept-id': 'G26...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,182.547405
3,"{'concept-type': 'granule', 'concept-id': 'G26...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,145.737147
4,"{'concept-type': 'granule', 'concept-id': 'G26...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,98.771830
...,...,...,...
83,"{'concept-type': 'granule', 'concept-id': 'G27...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,156.733950
84,"{'concept-type': 'granule', 'concept-id': 'G27...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,145.328370
85,"{'concept-type': 'granule', 'concept-id': 'G27...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,186.026457
86,"{'concept-type': 'granule', 'concept-id': 'G27...",{'TemporalExtent': {'RangeDateTime': {'Beginni...,97.142883


In [13]:
# look at the 'umm' column for the first row
wbd_hls_gdf['umm'].loc[0]

{'TemporalExtent': {'RangeDateTime': {'BeginningDateTime': '2023-05-04T16:31:32.101Z',
   'EndingDateTime': '2023-05-04T16:31:55.992Z'}},
 'GranuleUR': 'HLS.L30.T16RBT.2023124T163132.v2.0',
 'AdditionalAttributes': [{'Name': 'LANDSAT_PRODUCT_ID',
   'Values': ['LC08_L1TP_022039_20230504_20230504_02_RT',
    'LC08_L1TP_022040_20230504_20230504_02_RT']},
  {'Name': 'CLOUD_COVERAGE', 'Values': ['21']},
  {'Name': 'MGRS_TILE_ID', 'Values': ['16RBT']},
  {'Name': 'SPATIAL_COVERAGE', 'Values': ['57']},
  {'Name': 'SPATIAL_RESOLUTION', 'Values': ['30.0']},
  {'Name': 'SPATIAL_RESAMPLING_ALG', 'Values': ['Cubic Convolution']},
  {'Name': 'HLS_PROCESSING_TIME', 'Values': ['2023-05-06T06:52:48Z']},
  {'Name': 'SENSING_TIME',
   'Values': ['2023-05-04T16:31:32.1012330Z', '2023-05-04T16:31:55.9922730Z']},
  {'Name': 'HORIZONTAL_CS_NAME',
   'Values': ['UTM, WGS84, UTM ZONE 15', 'UTM, WGS84, UTM ZONE 15']},
  {'Name': 'ULX', 'Values': ['199980.0']},
  {'Name': 'ULY', 'Values': ['3300000.0']},
  {'N

In [14]:
# isolate the granule ID (UR) for the first row
row1 = list(wbd_hls_gdf['umm'].loc[0].values())
row1[1]

'HLS.L30.T16RBT.2023124T163132.v2.0'

In [15]:
# isolate the datetime for the first row
row1[0]


{'RangeDateTime': {'BeginningDateTime': '2023-05-04T16:31:32.101Z',
  'EndingDateTime': '2023-05-04T16:31:55.992Z'}}

In [16]:
# identify the keys for the first row
# the geometry, 'SpatialExtent' key is index 3
row1_keys = list(wbd_hls_gdf['umm'].loc[0].keys())
row1_keys


['TemporalExtent',
 'GranuleUR',
 'AdditionalAttributes',
 'SpatialExtent',
 'ProviderDates',
 'CollectionReference',
 'RelatedUrls',
 'DataGranule',
 'Platforms',
 'MetadataSpecification']

In [17]:
# isolate the SpatialExtent for the first row
# this is a dictionary
test = row1[3]
test

{'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -90.07372506,
        'Latitude': 28.80444525},
       {'Longitude': -89.57473392, 'Latitude': 28.81489352},
       {'Longitude': -89.30816409, 'Latitude': 29.81030157},
       {'Longitude': -90.10353675, 'Latitude': 29.79400715},
       {'Longitude': -90.07372506, 'Latitude': 28.80444525}]}}]}}}

In [18]:
# The first part of Elsa's loop to compile info abt each granule
# granule = the first result
granule = wbd_hls_results[0]
# look at just the metadata, umm, of the first result
info_dict = granule['umm']
# create a list of all points, lat/lon, for the first result from the umm column of the first result
points = (
    info_dict
    # select the first thing in the GPolygons list
    ['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]
    ['Boundary']['Points'])
# create the geometry (Polygon) of the first result using Polygon from shapely
# loop through all the points in our points list and get the lat/long for each one to create the geometry
geometry = Polygon(
    [(point['Longitude'], point['Latitude']) for point in points])

print(points)
print(geometry)

[{'Longitude': -90.07372506, 'Latitude': 28.80444525}, {'Longitude': -89.57473392, 'Latitude': 28.81489352}, {'Longitude': -89.30816409, 'Latitude': 29.81030157}, {'Longitude': -90.10353675, 'Latitude': 29.79400715}, {'Longitude': -90.07372506, 'Latitude': 28.80444525}]
POLYGON ((-90.07372506 28.80444525, -89.57473392 28.81489352, -89.30816409 29.81030157, -90.10353675 29.79400715, -90.07372506 28.80444525))


In [19]:
# open test granule
opened_granule = earthaccess.open([granule])

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
# look at opened_granule
# 15 different .tif files, each for a different band
opened_granule

[<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B07.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B06.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B02.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B01.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earth

In [21]:
# get tile ids and band numbers for the granule
# Compile a regular expression to search for metadata
# the r indicates this is a raw string and highlights all escaped characters (the dots), limits our need to escape special characters
uri_re = re.compile(
    r"HLS\.L30\.(?P<tile_id>T[0-9A-Z]+)\.(?P<date>\d+)T\d+\.v2\.0\."
    r"(?P<band_id>.+)\.tif"
)

#this yields a dictionary with the tile id, date, and band id
# 0 index above is B03, green band
# the date is the 124 day of 2023
uri_re.search(opened_granule[0].full_name).groupdict()

{'tile_id': 'T16RBT', 'date': '2023124', 'band_id': 'B03'}

Use above code to start trying to craft for loop(s)

In [ ]:
# Get granule metadata (ID (UR), datetime, and geometry)

## create empty lists to add metadata to
granule_ids = []
granule_datetimes = []
granule_geometries = []

## set up for loop, tqdm creates a progress bar
## for each row in the wbd_gls_results...
for row in tqdm(wbd_hls_results):
    metadata = row['umm']
    ## identify the granule_id and append it to the granule_ids list
    granule_id = metadata['GranuleUR']
    granule_ids.append(granule_id)

    ## identify the granule_datetime and append it to the granule_datetimes list
    granule_datetime = pd.to_datetime(metadata
        ['TemporalExtent']['RangeDateTime']['BeginningDateTime'])
    granule_datetimes.append(granule_datetime)

    ## identify the granule_geometry and append it to the granule_geometries list
    ## create a list of all points, lat/lon, for the first result from the umm column of the first result
    points = (
        metadata
        # select the first thing in the GPolygons list
        ['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]
        ['Boundary']['Points'])
    ## create the geometry (Polygon) of the first result using Polygon from shapely
    ## loop through all the points in our points list and get the lat/long for each one to create the geometry
    geometry = Polygon(
        [(point['Longitude'], point['Latitude']) for point in points])
    granule_geometries.append(geometry)

  0%|          | 0/88 [00:00<?, ?it/s]

,granule_ids,granule_datetimes,geometry
0,HLS.L30.T16RBT.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"POLYGON ((-90.07373 28.80445, -89.57473 28.814..."
1,HLS.L30.T15RYN.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"POLYGON ((-89.82661 28.80214, -89.79584 29.791..."
2,HLS.L30.T15RYP.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"POLYGON ((-89.79864 29.70348, -89.76644 30.692..."
3,HLS.L30.T16RBU.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"POLYGON ((-90.10082 29.70587, -89.33187 29.721..."
4,HLS.L30.T16RBT.2023132T163144.v2.0,2023-05-12 16:31:44.329000+00:00,"POLYGON ((-90.07373 28.80445, -89.56183 28.815..."
...,...,...,...
83,HLS.L30.T16RBU.2023292T163221.v2.0,2023-10-19 16:32:21.176000+00:00,"POLYGON ((-90.10082 29.70587, -89.31483 29.722..."
84,HLS.L30.T16RBU.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"POLYGON ((-90.10082 29.70587, -89.30583 29.722..."
85,HLS.L30.T15RYN.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"POLYGON ((-89.82661 28.80214, -89.79584 29.791..."
86,HLS.L30.T16RBT.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"POLYGON ((-90.07373 28.80445, -89.54924 28.815..."


In [ ]:
# get tile ids and band numbers for the granule
# Compile a regular expression to search for metadata
# the r indicates this is a raw string and highlights all escaped characters (the dots), limits our need to escape special characters
# def get_earthaccess_links(results):
#     uri_re = re.compile(
#         r"HLS\.L30\.(?P<tile_id>T[0-9A-Z]+)\.(?P<date>\d+)T\d+\.v2\.0\."
#         r"(?P<band_id>.+)\.tif"
# )
## create empty lists for results
files = []
files_metadata = []

## open each file in wbd_hls_results
for row in tqdm(wbd_hls_results):
    file = earthaccess.open([row])
    files.append(file)

    for opened_granule in files:
        uri_re = re.compile(
            r"HLS\.L30\.(?P<tile_id>T[0-9A-Z]+)\.(?P<date>\d+)T\d+\.v2\.0\."
            r"(?P<band_id>.+)\.tif")
        #this yields a dictionary with the tile id, date, and band id
        #why does this create such a long list? how to shorten it?
        file_metadata = uri_re.search(opened_granule[0].full_name)
        files_metadata.append(file_metadata)


  0%|          | 0/88 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

...

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

In [43]:
# check files list
files

[[<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif>,
  <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B07.tif>,
  <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B06.tif>,
  <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B02.tif>,
  <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T16RBT.2023124T163132.v2.0/HLS.L30.T16RBT.2023124T163132.v2.0.B01.tif>,
  <File-like object HTTPFileSystem, https://data.lpdaac

In [44]:
# check files_metadata list
files_metadata

[<re.Match object; span=(108, 150), match='HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T15RYN.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T15RYN.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T15RYP.2023124T163132.v2.0.VZA.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T15RYN.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T15RYP.2023124T163132.v2.0.VZA.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T16RBU.2023124T163132.v2.0.B11.tif'>,
 <re.Match object; span=(108, 150), match='HLS.L30.T16RBT.2023124T163132.v2.0.B03.tif'>,
 <re.Match object; sp

In [45]:
len(granule_ids)

88

In [46]:
len(files)

88

In [47]:
len(files_metadata)

3916

In [41]:
# Create dictionary to hold metadata lists
metadata_dict = ({'granule_ids' : granule_ids,
                  'granule_datetimes' : granule_datetimes,
                  'files' : files,
                  #'files_metadata' : files_metadata,
                  'geometry' : granule_geometries})

# Create GeoDataFrame out of metadata_dict
granule_gdf = gpd.GeoDataFrame(metadata_dict, crs="EPSG:4326")
granule_gdf

,granule_ids,granule_datetimes,files,geometry
0,HLS.L30.T16RBT.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.07373 28.80445, -89.57473 28.814..."
1,HLS.L30.T15RYN.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-89.82661 28.80214, -89.79584 29.791..."
2,HLS.L30.T15RYP.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-89.79864 29.70348, -89.76644 30.692..."
3,HLS.L30.T16RBU.2023124T163132.v2.0,2023-05-04 16:31:32.101000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.10082 29.70587, -89.33187 29.721..."
4,HLS.L30.T16RBT.2023132T163144.v2.0,2023-05-12 16:31:44.329000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.07373 28.80445, -89.56183 28.815..."
...,...,...,...,...
83,HLS.L30.T16RBU.2023292T163221.v2.0,2023-10-19 16:32:21.176000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.10082 29.70587, -89.31483 29.722..."
84,HLS.L30.T16RBU.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.10082 29.70587, -89.30583 29.722..."
85,HLS.L30.T15RYN.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-89.82661 28.80214, -89.79584 29.791..."
86,HLS.L30.T16RBT.2023300T163218.v2.0,2023-10-27 16:32:18.830000+00:00,"[<File-like object HTTPFileSystem, https://dat...","POLYGON ((-90.07373 28.80445, -89.54924 28.815..."


### Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the
granules you are downloading and make sure they line up with your
shapefile. You could also use a DataFrame, dictionary, or a custom
object to store this information.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each search result:
<ol type="1">
<li>Get the following information (HINT: look at the [‘umm’] values for
each search result):
<ul>
<li>granule id (UR)</li>
<li>datetime</li>
<li>geometry (HINT: check out the shapely.geometry.Polygon class to
convert points to a Polygon)</li>
</ul></li>
<li>Open the granule files. I recomment opening one granule at a time,
e.g. with (<code>earthaccess.open([result]</code>).</li>
<li>For each file (band), get the following information:
<ul>
<li>file handler returned from <code>earthaccess.open()</code></li>
<li>tile id</li>
<li>band number</li>
</ul></li>
</ol></li>
<li>Compile all the information you collected into a GeoDataFrame</li>
</ol></div></div>

In [7]:
# Loop through each granule

    # Get granule information

    # Get URL

    # Build metadata DataFrame rows

# Concatenate metadata DataFrame

### Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your
results using the `cached` decorator or by writing your own caching
code. I also recommend testing this step with one or two dates before
running the full computation.

This code should include at least one **function** including a
numpy-style docstring. A good place to start would be a function for
opening a single masked raster, applying the appropriate scale
parameter, and cropping.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each granule:
<ol type="1">
<li><p>Open the Fmask band, crop, and compute a quality mask for the
granule. You can use the following code as a starting point, making sure
that <code>mask_bits</code> contains the quality bits you want to
consider: ```python # Expand into a new dimension of binary bits bits =
( np.unpackbits(da.astype(np.uint8), bitorder=‘little’)
.reshape(da.shape + (-1,)) )</p>
<p># Select the required bits and check if any are flagged mask =
np.prod(bits[…, mask_bits]==0, axis=-1) ```</p></li>
<li><p>For each band that starts with ‘B’:</p>
<ol type="1">
<li>Open the band, crop, and apply the scale factor</li>
<li>Name the DataArray after the band using the <code>.name</code>
attribute</li>
<li>Apply the cloud mask using the <code>.where()</code> method</li>
<li>Store the DataArray in your data structure (e.g. adding a
GeoDataFrame column with the DataArray in it. Note that you will need to
remove the rows for unused bands)</li>
</ol></li>
</ol></li>
</ol></div></div>

In [9]:
# Loop through each image

    # Open granule cloud cover

    # Compute cloud mask

    # Loop through each spectral band

        # Open, crop, and mask the band

        # Add the DataArray to the metadata DataFrame row

    # Reassemble the metadata DataFrame

### Merge and Composite Data

You will notice for this watershed that: 1. The raster data for each
date are spread across 4 granules 2. Any given image is incomplete
because of clouds

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li><p>For each band:</p>
<ol type="1">
<li><p>For each date:</p>
<ol type="1">
<li>Merge all 4 granules</li>
<li>Mask any negative values created by interpolating from the nodata
value of -9999 (<code>rioxarray</code> should account for this, but
doesn’t appear to when merging. If you leave these values in they will
create problems down the line)</li>
</ol></li>
<li><p>Concatenate the merged DataArrays along a new date
dimension</p></li>
<li><p>Take the mean in the date dimension to create a composite image
that fills cloud gaps</p></li>
<li><p>Add the band as a dimension, and give the DataArray a
name</p></li>
</ol></li>
<li><p>Concatenate along the band dimension</p></li>
</ol></div></div>

In [11]:
# Merge and composite and image for each band

        # Merge granules for each date

        # Mask negative values

    # Composite images across dates

## STEP 4: K-MEANS

Cluster your data by spectral signature using the k-means algorithm.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Convert your DataArray into a <strong>tidy</strong> DataFrame of
reflectance values (hint: check out the <code>.to_dataframe()</code> and
<code>.unstack()</code> methods)</li>
<li>Filter out all rows with no data (all 0s or any N/A values)</li>
<li>Fit a k-means model. You can experiment with the number of groups to
find what works best.</li>
</ol></div></div>

In [ ]:
# Convert spectral DataArray to a tidy DataFrame
# Create a column for each of the bands
## model_df = reflectance_df.to_dataframe().reflectance.unstack('band)

# drop bands 10 and 11 and any rows w/ NaN values
## model_df = model_df.drop(columns = [10,11]).drop(na)

# Check that the ranges make sense (should be from 0-1) by checking min and max values
## min_values = model_df.min()
## max_values = model_df.max()
## print(min_values)
## pring(max_values)

# Initialize k means
## k_means = KMeans(n_clusters = 5)

# Fit model and predict
## prediction = k_means.fit_predict(model_df.values)

# Add cluster label to df
## model_df['clusters'] = prediction

# Running the fit and predict functions at the same time.
# We can do this since we don't have target data.

# Add the predicted values back to the model DataFrame

## STEP 5: PLOT

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Create a plot that shows the k-means clusters next to an RGB image of
the area. You may need to brighten your RGB image by multiplying it by
10. The code for reshaping and plotting the clusters is provided for you
below, but you will have to create the RGB plot yourself!</p>
<p>So, what is <code>.sortby(['x', 'y'])</code> doing for us? Try the
code without it and find out.</p></div></div>

In [ ]:
# make dataarray w/ bands to use for rgb
## rgb = reflectance_da.sel(band=[4, 3, 2])

# hvplot has a rgb plot method. this will not give us a helpful plot b/c the values arent what the rgb plot expects
# (
#     rgb.hvplot.rgb(y = 'y',
#                x = 'x',
#                bands = 'band',
#                data_aspect = 1,
#                xaxis = None, yaxis = None)
#                )

# Want image pixel values to range from 0-255
# Then convert to unsigned 8-bit integers (data type for images)
# don't include nan values in calculation
# This still yields a very dark plot
## rgb_uint8 = (rgb * 255).astype(np.uint8.where(rgb!=np.nan))

# We can increase the brightness of the image
# After plotting this we may get an error that the data values are too high.
## rgb_uint8_bright = (rgb_uint8 * 10)

# To fix the error, cap all pixel values at 255
# If value is 155, keep that value. if value is greater than 255, replace w/ 255
## rgb_sat = rgb_uint8_bright.where(rgb_uint8_bright < 255, 255)

In [15]:
# Plot the k-means clusters
(
    rgb_plot
    + 
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        cmap="Colorblind", aspect='equal') 
)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Don’t forget to interpret your plot!</p></div></div>

**YOUR PLOT HEADLINE AND DESCRIPTION HERE**